## Modèle Encodeur-Décodeur version Corrigé
Dans cette version, j'ai redonné mon précédent Notebook a ChatGPT 4o pour comprendre ce qui n'allait pas dans mon code. En fait le principal problème c'est que je ne séparais pas dès le début mes phrases françaises et anglaises et que je voulais faire mon modèle encodeur décodeur sur le resultat de mon embedding.


In [96]:
#On importe toutes les librairies nécessaires 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense 
#Input pour déclarer la forme du modèle
#Embedding pour pouvoir coder les phrases
#LSTM pour lire les phrases en français dans l'encodeur et générer la phrase anglaise mot par mot dans le décodeur
#Dense pour transformer le hidden state en probabilité en sortie du décodeur
from tensorflow.keras.preprocessing.text import one_hot # pour faire le mapping one-hot
from tensorflow.keras.preprocessing.sequence import pad_sequences # pour normaliser la dimension des vecteurs
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# --- Données sources ---
pairs = [
    ("je t'aime", "i love you"),
    ("j'aime les pâtes", "i love pasta"),
    ("bonjour", "hello"),
    ("je suis fatigué", "i am tired"),
]


On récupère les paramètres qu'on avait utilisé dans notre premier Notebook 

In [98]:
# Paramètres
vocab_size = 500 # taille du vocabulaire
embedding_dim = 10 # dimension du vecteur associé à un mot après embedding
latent_dim = 64 # dimension du hidden state
sent_length = 5  # nombre de mot max dans nos phrases

Ici on fait l'encodage one-hot et le padding. On a déjà séparré fr et en ici. 

In [100]:
# --- Tokenizer français (source)
tokenizer_fr = Tokenizer(num_words=vocab_size, oov_token="<unk>")
tokenizer_fr.fit_on_texts(french_sentences)
encoded_fr = tokenizer_fr.texts_to_sequences(french_sentences)
padded_fr = pad_sequences(encoded_fr, maxlen=sent_length, padding='pre')

# --- Tokenizer anglais (cible)
tokenizer_en = Tokenizer(num_words=vocab_size, oov_token="<unk>")
tokenizer_en.fit_on_texts(english_sentences)
encoded_en = tokenizer_en.texts_to_sequences(english_sentences)
padded_en = pad_sequences(encoded_en, maxlen=sent_length, padding='pre')



 On fait les decoder_input et decoder_targer ça nous permet d'avoir la structure de prédiction du dernier mot

In [102]:
# --- Décodeur : inputs décalés
decoder_input = padded_en[:, :-1]  # sans le dernier mot
decoder_target = padded_en[:, 1:]  # sans le premier mot

# Reshape pour sparse_categorical_crossentropy
#Ça va nous permettre de pouvoir donner à manger à notre fonction de perte 
decoder_target = decoder_target.reshape(decoder_target.shape[0], decoder_target.shape[1], 1)


In [104]:
# --- Modèle encodeur-décodeur ---
# Encodeur
encoder_inputs = Input(shape=(sent_length,))
enc_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs) # permet d'encoder chaque mot en vecteur de taille = 10
_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb) # permet de récupérer les états cachés (sortie réelle) et cellules (mémoire long terme) 
encoder_states = [state_h, state_c] # on stock les états pour les passer au décodeurs

# Décodeur
decoder_inputs = Input(shape=(sent_length - 1,))
dec_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modèle final
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 5, 10)     │      5,000 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, 4, 10)     │      5,000 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 64),      │     19,200 │ embedding_9[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 4, 64),   │     19,200 │ embedding_10[0][… │
│                     │ (None, 64),       │            │ lstm_4[0][1],     │
│                     │ (None, 64)]       │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 4, 500)    │     32,500 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 80,900 (316.02 KB)

 Trainable params: 80,900 (316.02 KB)

 Non-trainable params: 0 (0.00 B)

Le nombre d'Epoch correspond au nombre de fois où les données passent dans le modèle. Plus on l'augmente plus il va voir passer nos données mais plus il risque de connaître par coeur spécialement nos données.

In [106]:
# --- Entraînement ---
model.fit([padded_fr, decoder_input], decoder_target, epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 6.2134
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4375 - loss: 6.2089
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4375 - loss: 6.2043
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3750 - loss: 6.1995
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3750 - loss: 6.1944
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3750 - loss: 6.1888
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3750 - loss: 6.1827
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3750 - loss: 6.1760
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3750 - loss: 6.1686
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3750 - loss: 6.1602
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3750 - loss: 6.1509
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3750 - 

### Inférence
On ajoute une partie inférentielle dans le code afin d'essayer de traduire une phrase. On va donc construire nos modèles pour l'inférence

In [108]:
# --- Modèle encodeur pour l’inférence ---
encoder_model = Model(encoder_inputs, encoder_states)

# --- Modèle décodeur pour l’inférence ---
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Reutilisation de la couche Embedding
dec_emb2 = decoder_embedding_layer(decoder_inputs)  # il faut nommer ton layer d’embedding au-dessus
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [110]:
decoder_embedding_layer = Embedding(vocab_size, embedding_dim)
dec_emb = decoder_embedding_layer(decoder_inputs)


In [112]:
def decode_sequence(input_seq):
    # 1. Encoder la phrase française
    states_value = encoder_model.predict(input_seq)

    # 2. Commencer avec le token de début (ici 0 si pas de <sos>)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 0  # ou tokenizer_en.word_index['<sos>']

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = index2word_en.get(sampled_token_index, '?')  # à définir

        decoded_sentence.append(sampled_word)

        if sampled_word == '<eos>' or len(decoded_sentence) > sent_length:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(decoded_sentence)


In [114]:
# Phrase en français à traduire
test_sentence = "je t'aime"
test_encoded = tokenizer_fr.texts_to_sequences([test_sentence])
test_padded = pad_sequences(test_encoded, maxlen=sent_length, padding='pre')

# Traduction
print("Traduction :", decode_sequence(test_padded))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Traduction : ? ? ? ? ? ?


Problème avec l'utilisation du one hot. même en modifiant en tokenizer ça ne fonctionne pas.